In [1]:
import cv2
import numpy as np

# 读取图像
image = cv2.imread('dog.jpg',cv2.IMREAD_GRAYSCALE)

# 应用高斯滤波
sigma1 = 1
sigma2 = 2
blurred_image1 = cv2.GaussianBlur(image, (5, 5), sigma1)
blurred_image2 = cv2.GaussianBlur(image, (5, 5), sigma2)
DoG = blurred_image2 - blurred_image1


# 显示结果
cv2.imshow('Original', image)
cv2.imshow('Blurred', DoG)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [4]:
class DoGFilter(nn.Module):
    def __init__(self, in_channels, kernel_size,sigma1,sigma2, stride=1, padding=0):
        super(DoGFilter, self).__init__()
        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.sigma1 = sigma1
        self.sigma2 = sigma2
        # initiate
        self.weight1 = nn.Parameter(torch.randn(1, in_channels, kernel_size, kernel_size))
        self.weight2 = nn.Parameter(torch.randn(1, in_channels, kernel_size, kernel_size))
        
        #create gaussin kernel 
    def DoG_kernel(self, sigma1, sigma2, size):
        size = int(size) // 2
        x, y = np.mgrid[-size:size+1, -size:size+1]
        normal1 = 1 / (2.0 * np.pi * sigma1**2)
        normal2 = 1 / (2.0 * np.pi * sigma2**2)
        g1 = np.exp(-((x**2 + y**2) / (2.0*sigma1**2))) * normal1
        g2 = np.exp(-((x**2 + y**2) / (2.0*sigma2**2))) * normal2
        # transfer to Tensor
        return torch.from_numpy(g1).float(), torch.from_numpy(g2).float()           
        
    def forward(self, x, sigma1, sigma2):
        # create gaussin kernel
        self.weight1, self.weight2 = self.DoG_kernel(self.sigma1, self.sigma2, self.kernel_size)
        x1 = F.conv2d(x, self.weight1, self.bias, self.stride, self.padding)
        x2 = F.conv2d(x, self.weight2, self.bias, self.stride, self.padding)
        x_on = x1 - x2 #on center filter
        x_off = x2 - x1#off center filter
        x = torch.cat((x_on, x_off), dim=1)
        return x
          

In [11]:
image = cv2.imread('dog.jpg',cv2.IMREAD_GRAYSCALE)
dogfilter = DoGFilter(in_channels=1, sigma1=1,sigma2=2,kernel_size=5)
picture = dogfilter(sigma1=1,sigma2=2)(image)
cv2.imshow('Original', picture)

TypeError: forward() missing 1 required positional argument: 'x'

In [ ]:
import torch

from norse.torch.functional.lif import LIFParameters
from norse.torch.module.leaky_integrator import LILinearCell
from norse.torch.module.lif import LIFCell


class ConvNet_STDP(torch.nn.Module):
    """
    A convolutional network with LIF dynamics

    Arguments:
        num_channels (int): Number of input channels
        feature_size (int): Number of input features
        method (str): Threshold method
    """

    def __init__(
        self, num_channels=1, feature_size=28, method="super", dtype=torch.float
    ):
        super(ConvNet_STDP, self).__init__()
        
        self.features = int(((feature_size - 4) / 2 - 4) / 2)
        self.conv1 = torch.nn.Conv2d(num_channels, 30, 5, 1)
        self.conv2 = torch.nn.Conv2d(30, 100, 5, 1)
        self.fc1 = torch.nn.Linear(self.features * self.features * 50, 500)
        self.out = LILinearCell(500, 10)
        self.lif0 = LIFCell(
            p=LIFParameters(method=method, alpha=torch.tensor(100.0),v_th= 15),
        )
        self.lif1 = LIFCell(
            p=LIFParameters(method=method, alpha=torch.tensor(100.0),v_th= 10),
        )
        self.lif2 = LIFCell(p=LIFParameters(method=method, alpha=torch.tensor(100.0)))
        self.dtype = dtype
        self.dogfilter = DoGFilter(in_channels=1, sigma1=1,sigma2=2,kernel_size=5)

    def forward(self, x):
        seq_length = x.shape[0]
        batch_size = x.shape[1]

        # specify the initial states
        s0, s1, s2, s3, so = None, None, None, None,None

        voltages = torch.zeros(
            seq_length, batch_size, 10, device=x.device, dtype=self.dtype
        )
        ConvNet_STDP
        for ts in range(seq_length):
            z = self.dogfilter(x[ts, :])
            z, s0 = self.lif0(z, s0)
            z = self.conv1(z)
            z, s1 = self.lif0(z, s1)
            z = torch.nn.functional.max_pool2d(z, kernel_size = 2, stride = 2)
            z = 10 * self.conv2(z)
            z, s2 = self.lif1(z, s2)
            z = torch.nn.AdaptiveMaxPool2d(z)
            z = z.view(-1, 100)
            z = self.fc1(z)
            z, s3 = self.lif2(z, s3)
            v, so = self.out(torch.nn.functional.relu(z), so)
            voltages[ts, :, :] = v
        return voltages


